Here we are creating a new verion of the sample for the test data that tries to improve the iter-rater reliabiliy agreements on the sentiment of the answers/texts, by only keeping those sentences that, in each answer/text, are subjective.

We do it by using TextBlob and the user-defined functions remove_objective_sents().

### 1. Imports and Set Up

In [36]:
import os
import pandas as pd
import numpy as np

#### Set up working directory

cwd = os.chdir('/Users/alessia/Documents/DataScience/NLP_Project/Outputs')

In [37]:
pd.set_option('display.max_colwidth', -1)

### 2. Get Data

In [38]:
# Read in data (note header is spread over two rows)

testdata_sample = pd.read_csv("sa_q1_sample_2.csv")

### 3. Import basic NLP functions

In [39]:
cwd = os.chdir('/Users/alessia/Documents/DataScience/textconsultations/')

In [40]:
os.listdir()

['nlpfunctions', 'tutorial', 'README.md', '.git']

In [41]:
os.listdir('nlpfunctions');

In [42]:
import nlpfunctions.basic_NLP_functions as b_nlp

### 4. Remove objective sentences from each answer/text

In [43]:
testdata_sample.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Respondent ID', 'Q1_census_methods',
       'Are you responding on behalf of an organisation, or as an individual?Response',
       'PublicSector', 'PrivateSector', 'OtherSectors', 'sent_tok_text',
       'SA_scores_sents', 'mean_SA_scores', 'SA_polarity',
       'textblob_SA_score_sents', 'textblob_mean_SA_score',
       'textblob_polarity'],
      dtype='object')

In [46]:
testdata_sample['sent_tok_text'] = testdata_sample['Q1_census_methods'].apply(lambda x: b_nlp.sent_tokenise_df(x))

In [47]:
testdata_sample['subj_score_sents'] = testdata_sample['sent_tok_text'].apply(lambda x: b_nlp.get_subjectivity_df(x))

In [48]:
testdata_sample['Q1_only_subj_sents'] = testdata_sample['sent_tok_text'].apply(lambda x: b_nlp.remove_objective_sents_df(x))

In [50]:
testdata_sample[['subj_score_sents', 'SA_polarity','Q1_only_subj_sents']];

### 5. Recalculate Vader and TextBlob polarity scores for the subjective-only text

#### Vader

In [51]:
testdata_sample['SAscore_only_subj_sents'] = testdata_sample['Q1_only_subj_sents'].apply(lambda x: b_nlp.get_sentiment_score_df(x))

In [52]:
testdata_sample['meanSAscore_only_subj_sents'] = testdata_sample['SAscore_only_subj_sents'].apply(lambda x: np.mean(x))

In [53]:
testdata_sample['SA_polarity_only_subj_sents'] = testdata_sample['meanSAscore_only_subj_sents'].apply(lambda x: 'pos' if x > 0 else 'neg' if x < 0 else "")

#### TextBlob

In [54]:
testdata_sample['TB_SAscore_only_subj_sents'] = testdata_sample['Q1_only_subj_sents'].apply(lambda x: b_nlp.get_textblob_sentiment_score_df(x))

In [59]:
testdata_sample['TB_meanSAscore_only_subj_sents'] = testdata_sample['TB_SAscore_only_subj_sents'].apply(lambda x: np.mean(x))

In [60]:
testdata_sample['TB_SA_polarity_only_subj_sents'] = testdata_sample['TB_meanSAscore_only_subj_sents'].apply(lambda x: 'pos' if x > 0 else 'neg' if x < 0 else "")

#### Agreement

In [34]:
#testdata_sample.iloc[:, -5:]

In [63]:
# Checks
testdata_sample.head(10);

In [65]:
pd.crosstab(testdata_sample['SA_polarity_only_subj_sents'], testdata_sample['TB_SA_polarity_only_subj_sents'])

TB_SA_polarity_only_subj_sents,,neg,pos
SA_polarity_only_subj_sents,,,
,62,5,9
neg,0,14,34
pos,4,6,66


TB tends to classify more texts as positive than TextBlob.
Now 62 texts were removed from the sample as conaining only objective sentences.